# Импортируем библеотеки 

In [1]:
import cv2 
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, Dense, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.preprocessing import image

In [2]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

Добавляем слои

In [3]:
model = Sequential()
model.add(Conv2D(75, (3, 3), strides=1, padding='same', activation='relu', input_shape=(32, 32, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2), strides=2, padding='same'))
model.add(Conv2D(50, (3, 3), strides=1, padding='same', activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2), strides=2, padding='same'))
model.add(Conv2D(25, (3, 3), strides=1, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2), strides=2, padding='same'))
model.add(Flatten())
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=10, activation='softmax'))

model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [4]:
model.load_weights('model_cnn.h5')

# Запуск камеры

In [5]:
cap = cv2.VideoCapture(0)

Функции обработки

In [6]:
def create_image_from_landmarks(landmarks, img_size):
    img = np.zeros((img_size[0], img_size[1], 3), dtype=np.uint8)

    # Определите область, где располагается рука
    hand_area = get_hand_area(landmarks, img_size)

    for landmark in landmarks:
        x = int(landmark.x * img_size[0])
        y = int(landmark.y * img_size[1])

        # Отрисовать точку на изображении только внутри области руки
        if is_point_inside_hand_area(x, y, 0, hand_area):  # z-координата в данном случае не используется
            img = cv2.circle(img, (x, y), 3, (255, 255, 255), -1)

    return img

def get_hand_area(landmarks, img_size):
    # Простой прямоугольник, охватывающий включенные точки
    active_points_x = [i for i, value in enumerate(landmarks) if value.x == 1]
    active_points_y = [i for i, value in enumerate(landmarks) if value.y == 1]
    active_points_z = [i for i, value in enumerate(landmarks) if value.z == 1]

    min_x = min(active_points_x) if active_points_x else 0
    min_y = min(active_points_y) if active_points_y else 0
    min_z = min(active_points_z) if active_points_z else 0
    max_x = max(active_points_x) if active_points_x else img_size[0]
    max_y = max(active_points_y) if active_points_y else img_size[1]
    max_z = max(active_points_z) if active_points_z else 1  # Предполагается, что z в диапазоне от 0 до 1

    return (min_x, min_y, min_z, max_x, max_y, max_z)

def is_point_inside_hand_area(x, y, z, hand_area):
    min_x, min_y, min_z, max_x, max_y, max_z = hand_area
    return min_x <= x <= max_x and min_y <= y <= max_y and min_z <= z <= max_z

def draw_hand_landmarks(frame, landmarks, img_size):
    for landmark in landmarks:
        x = int(landmark.x * img_size[0])
        y = int(landmark.y * img_size[1])
        cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)  # Отрисовка кружка вокруг точки
    return frame

In [7]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Обработка кадра Mediapipe
    results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    if results.multi_hand_landmarks:
        landmarks = results.multi_hand_landmarks[0].landmark
        print(landmarks)
        # Преобразование landmarks в изображение
        img_size = (32, 32)  # Настройте размер изображения под требования вашей нейронной сети
        img_array = create_image_from_landmarks(landmarks, img_size)
        
        # Преобразование изображения в вектор
        img_array = np.expand_dims(img_array, axis=0)
        
        # Отрисовка точек на руке
        frame = draw_hand_landmarks(frame, landmarks, (frame.shape[1], frame.shape[0]))
        
        # Предсказание с помощью вашей нейронной сети
        predicted_number = model.predict(img_array)

        # Отображение результата
        cv2.putText(frame, f"Number: {np.argmax(predicted_number, axis=1)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Отображение кадра
    cv2.imshow('Gesture Recognition', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Очистка ресурсов
cap.release()
cv2.destroyAllWindows()